Если прогнозы, полученные с помощью прямой линии, сравнить с
прогнозами KNeighborsRegressor (рис. 2.10), использование линии
регрессии для получения прогнозов кажется очень строгим. Похоже, что
62
все мелкие детали данных не учитываются. В некотором смысле это
верно. Мы видвигаем сильное (и в некоторой степени нереальное)
предположение, что наша целевая переменная у является линейной
комбинацией признаков. Однако анализ одномерных данных дает
несколько искаженную картину. Для наборов данных с большим
количеством признаков линейные модели могут быть очень полезны. В
частности, если у вас количество признаков превышает количество точек
данных для обучения, любую целевую переменную у можно прекрасно
смоделировать (на обучающей выборке) в виде линейной функции.

Существует различные виды линейных моделей для регрессии.
Различие между этими моделями заключается в способе оценивания
параметров модели w и b по обучающим данным и контроле сложности
модели. Теперь мы рассмотрим наиболее популярные линейные модели
для регрессии.

# Линейный регрессия (метод наименьших квадратов)

Линейная регрессия находит параметры
w и b, которые минимизируют среднеквадратическую ошибку (mean
squared error) между спрогнозированными и фактическими ответами у в
обучающем наборе

In [22]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, random_state=24)

Параметры «наклона» (w), также называемые весами или
коэффициентами (coefficients), хранятся в атрибуте coef_, тогда как
сдвиг (offset) или константа (intercept), обозначаемая как b, хранится в
атрибуте intercept_:

In [18]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

Правильность на обучающем наборе: 0.75
Правильность на тестовом наборе: 0.69


In [19]:
print("lr.coef_: \n{}".format(lr.coef_))
print("lr.intercept_: \n{}".format(lr.intercept_))

lr.coef_: 
[-5.10594462e-02  4.17629732e-02 -6.71758512e-03  2.97423286e+00
 -1.43617375e+01  4.29486456e+00 -7.21470856e-03 -1.42539988e+00
  2.48641697e-01 -1.23390623e-02 -8.58249084e-01  1.04412872e-02
 -4.97967036e-01]
lr.intercept_: 
30.175131465517868


Атрибут intercept_ - это всегда отдельное число с плавающей точкой,
тогда как атрибут coef_ - это массив NumPy, в котором каждому
элементу соответствует входной признак.

Для этого одномерного массива данных
опасность переобучения невелика, поскольку модель очень проста (или
строга). Однако для высокоразмерных наборов данных (наборов данных
с большим количеством признаков) линейные модели становятся более
сложными и существует более высокая вероятность переобучения.

Давайте посмотрим, как LinearRegression сработает на более сложном
наборе данных, например, на наборе Boston Housing. Вспомним, что этот
набор данных имеет 506 примеров (наблюдений) и 105 производных
признаков.

In [43]:
import mglearn
X, y = mglearn.datasets.load_extended_boston()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lr = LinearRegression().fit(X_train, y_train)

C:\Users\Asus_Shamil\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets inclu

In [44]:
print("Правильность на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))

Правильность на обучающем наборе: 0.95
Правильность на тестовом наборе: 0.61


Это несоответствие между правильностью на обучающем наборе и
правильностью на тестовом наборе является явным признаком
переобучения и поэтому мы должны попытаться найти модель, которая
позволит нам контролировать сложность

# Гребневая регрессия (L2 регуляризация)

In [46]:
from sklearn.linear_model import Ridge
ridge = Ridge().fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))

Правильность на обучающем наборе: 0.89
Правильность на тестовом наборе: 0.75


Увеличение alpha заставляет коэффициенты сжиматься
до близких к нулю значений, что снижает качество работы модели на
обучающем наборе, но может улучшить ее обобщающую способность.
Например:

In [47]:
ridge10 = Ridge(alpha=10).fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(ridge10.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.2f}".format(ridge10.score(X_test, y_test)))


Правильность на обучающем наборе: 0.79
Правильность на тестовом наборе: 0.64


# Лассо (L1 регуляризация)

Альтернативой Ridge как метода регуляризации линейной регрессии
является Lasso. Как и гребневая регрессия, лассо также сжимает
коэффициенты до близких к нулю значений, но несколько иным
способом, называемым L1 регуляризацией.13 Результат L1 регуляризации
заключается в том, что при использовании лассо некоторые
коэффициенты становятся равны точно нулю. 

In [50]:
from sklearn.linear_model import Lasso
import numpy as np

In [52]:
lasso = Lasso(alpha=0.01, max_iter=100000).fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.2f}".format(lasso.score(X_train, y_train)))
print("Правильность на контрольном наборе: {:.2f}".format(lasso.score(X_test, y_test)))
print("Количество использованных признаков: {}".format(np.sum(lasso.coef_ != 0)))

Правильность на обучающем наборе: 0.90
Правильность на контрольном наборе: 0.77
Количество использованных признаков: 33
